In [2]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:


# Define your dataset directory
dataset_dir = 'dataset'

# Define image size and batch size
img_height, img_width = 640, 640
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting data into training and validation sets
)

# Load and prepare training data
train_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load and prepare validation data
val_data = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 1040 images belonging to 10 classes.
Found 255 images belonging to 10 classes.


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define your dataset directory and other parameters
dataset_dir = 'dataset'
img_height, img_width = 640, 640
num_classes = 10  # Specify the number of classes in your dataset

# Define the CNN-LSTM model
inputs = layers.Input(shape=(img_height, img_width, 3))  # Input shape for the images

# CNN feature extractor
cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten()
])
cnn_outputs = cnn(inputs)

# Reshape CNN output for LSTM input
cnn_outputs = layers.Reshape((1, cnn_outputs.shape[1]))(cnn_outputs)

# LSTM layer
lstm_out = layers.LSTM(64)(cnn_outputs)

# Dense layers for classification
outputs = layers.Dense(num_classes, activation='softmax')(lstm_out)

# Create the model
model = models.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 640, 640, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 184832)            240832    
                                                                 
 reshape_1 (Reshape)         (None, 1, 184832)         0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                47333632  
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 47575114 (181.48 MB)
Trainable params: 47575114 (181.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Train the model
history = model.fit(
    train_data,
    epochs=10,
    batch_size=batch_size,
    validation_data=val_data
)


Epoch 1/10
33/33 [==============================] - 190s 6s/step - loss: 2.3334 - accuracy: 0.1212 - val_loss: 2.2646 - val_accuracy: 0.1098
Epoch 2/10
33/33 [==============================] - 176s 5s/step - loss: 2.0353 - accuracy: 0.2337 - val_loss: 1.6912 - val_accuracy: 0.3922
Epoch 3/10
33/33 [==============================] - 174s 5s/step - loss: 1.4945 - accuracy: 0.4673 - val_loss: 1.2953 - val_accuracy: 0.5529
Epoch 4/10
33/33 [==============================] - 185s 6s/step - loss: 1.2307 - accuracy: 0.5606 - val_loss: 1.2494 - val_accuracy: 0.4784
Epoch 5/10
33/33 [==============================] - 1202s 37s/step - loss: 1.0842 - accuracy: 0.6240 - val_loss: 0.9615 - val_accuracy: 0.6078
Epoch 6/10
33/33 [==============================] - 177s 5s/step - loss: 0.8718 - accuracy: 0.7183 - val_loss: 0.9377 - val_accuracy: 0.6706
Epoch 7/10
33/33 [==============================] - 176s 5s/step - loss: 0.8982 - accuracy: 0.6798 - val_loss: 0.8346 - val_accuracy: 0.7059
Epoch 8/10
